####Nurul Salsabila Syam - H071191054

source: https://d2l.ai/chapter_deep-learning-computation/index.html

#5. Deep Learning Computation using Tensorflow

In [1]:
import tensorflow as tf
from tensorflow import keras

##5.1 Layer and Blocks

Kode berikut menghasilkan network dengan satu fully-connected hidden layer dengan 256 unit dan fungsi aktivasi ReLU, diikuti dengan fully-connected output layer dengan 10 unit

In [2]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])

X = tf.random.uniform((2, 20))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.08287849, -0.30346844, -0.04760871, -0.26502824, -0.09242874,
        -0.13416907,  0.03439464, -0.10076965, -0.11438741,  0.09525585],
       [-0.24091437, -0.3161515 , -0.07254852, -0.21608725, -0.10917594,
        -0.30933452,  0.00992046,  0.02660128, -0.16922517,  0.23943578]],
      dtype=float32)>

###5.1.1 A Custom Block

In [3]:
class MLP(tf.keras.Model):
    # Declare a layer with model parameters. Here, we declare two fully
    # connected layers
    def __init__(self):
        # Call the constructor of the `MLP` parent class `Model` to perform
        # the necessary initialization. In this way, other function arguments
        # can also be specified during class instantiation, such as the model
        # parameters, `params` (to be described later)
        super().__init__()
        # Hidden layer
        self.hidden = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(units=10)  # Output layer

    # Define the forward propagation of the model, that is, how to return the
    # required model output based on the input `X`
    def call(self, X):
        return self.out(self.hidden((X)))

In [4]:
net = MLP()
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.29849792,  0.13171503, -0.00290155,  0.12456489, -0.1641078 ,
         0.20013893, -0.00719571, -0.04755875, -0.10638353, -0.0863733 ],
       [ 0.08399504,  0.38717115, -0.07206587,  0.22723827, -0.01109883,
         0.09119499,  0.1832793 , -0.24725065, -0.11943671,  0.06656247]],
      dtype=float32)>

###5.1.2 The Sequential Block

In [5]:
class MySequential(tf.keras.Model):
    def __init__(self, *args):
        super().__init__()
        self.modules = []
        for block in args:
            # Here, `block` is an instance of a `tf.keras.layers.Layer`
            # subclass
            self.modules.append(block)

    def call(self, X):
        for module in self.modules:
            X = module(X)
        return X

In [6]:
net = MySequential(
    tf.keras.layers.Dense(units=256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.18955854, -0.2548249 , -0.01406711,  0.19364023,  0.18210216,
        -0.0262994 ,  0.12603891,  0.496101  , -0.26568264,  0.26428166],
       [-0.18273705, -0.032057  ,  0.10518166,  0.22461691,  0.18989684,
         0.01142827,  0.26264113,  0.590247  , -0.10567659,  0.26649332]],
      dtype=float32)>

###5.1.3 Executing Code in the Forward Propagation Function

In [7]:
class FixedHiddenMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        # Random weight parameters created with `tf.constant` are not updated
        # during training (i.e., constant parameters)
        self.rand_weight = tf.constant(tf.random.uniform((20, 20)))
        self.dense = tf.keras.layers.Dense(20, activation=tf.nn.relu)

    def call(self, inputs):
        X = self.flatten(inputs)
        # Use the created constant parameters, as well as the `relu` and
        # `matmul` functions
        X = tf.nn.relu(tf.matmul(X, self.rand_weight) + 1)
        # Reuse the fully-connected layer. This is equivalent to sharing
        # parameters with two fully-connected layers
        X = self.dense(X)
        # Control flow
        while tf.reduce_sum(tf.math.abs(X)) > 1:
            X /= 2
        return tf.reduce_sum(X)

In [8]:
net = FixedHiddenMLP()
net(X)

<tf.Tensor: shape=(), dtype=float32, numpy=0.50228256>

In [9]:
class NestMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.net = tf.keras.Sequential()
        self.net.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
        self.net.add(tf.keras.layers.Dense(32, activation=tf.nn.relu))
        self.dense = tf.keras.layers.Dense(16, activation=tf.nn.relu)

    def call(self, inputs):
        return self.dense(self.net(inputs))

chimera = tf.keras.Sequential()
chimera.add(NestMLP())
chimera.add(tf.keras.layers.Dense(20))
chimera.add(FixedHiddenMLP())
chimera(X)

<tf.Tensor: shape=(), dtype=float32, numpy=0.8812042>

##5.2 Parameter Management

In [10]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4, activation=tf.nn.relu),
    tf.keras.layers.Dense(1),
])

X = tf.random.uniform((2, 4))
net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-0.31128165],
       [-0.11215455]], dtype=float32)>

###5.2.1 Parameter Access

In [11]:
print(net.layers[2].weights)

[<tf.Variable 'dense_13/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[ 1.0414107 ],
       [-0.1369335 ],
       [-0.41871077],
       [-0.84066296]], dtype=float32)>, <tf.Variable 'dense_13/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


####5.2.1.1 Targeted Parameters

In [12]:
print(type(net.layers[2].weights[1]))
print(net.layers[2].weights[1])
print(tf.convert_to_tensor(net.layers[2].weights[1]))

<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'>
<tf.Variable 'dense_13/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>
tf.Tensor([0.], shape=(1,), dtype=float32)


####5.2.1.2 All Parameters at Once

In [13]:
print(net.layers[1].weights)
print(net.get_weights())

[<tf.Variable 'dense_12/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[-0.6212324 , -0.84532696,  0.15466255, -0.48657602],
       [-0.13326514, -0.42603317, -0.7168213 ,  0.59406894],
       [-0.5971612 , -0.23710406,  0.59324557, -0.69837844],
       [ 0.47262937,  0.04241276, -0.40341285,  0.5221724 ]],
      dtype=float32)>, <tf.Variable 'dense_12/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]
[array([[-0.6212324 , -0.84532696,  0.15466255, -0.48657602],
       [-0.13326514, -0.42603317, -0.7168213 ,  0.59406894],
       [-0.5971612 , -0.23710406,  0.59324557, -0.69837844],
       [ 0.47262937,  0.04241276, -0.40341285,  0.5221724 ]],
      dtype=float32), array([0., 0., 0., 0.], dtype=float32), array([[ 1.0414107 ],
       [-0.1369335 ],
       [-0.41871077],
       [-0.84066296]], dtype=float32), array([0.], dtype=float32)]


####5.2.1.3 Collecting Parameters from Nested Blocks

In [14]:
def block1(name):
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(4, activation=tf.nn.relu)],
        name=name)

def block2():
    net = tf.keras.Sequential()
    for i in range(4):
        # Nested here
        net.add(block1(name=f'block-{i}'))
    return net

rgnet = tf.keras.Sequential()
rgnet.add(block2())
rgnet.add(tf.keras.layers.Dense(1))
rgnet(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.12701169],
       [0.07473375]], dtype=float32)>

In [15]:
print(rgnet.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_5 (Sequential)   (2, 4)                    80        
                                                                 
 dense_18 (Dense)            (2, 1)                    5         
                                                                 
Total params: 85
Trainable params: 85
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
rgnet.layers[0].layers[1].layers[1].weights[1]

<tf.Variable 'dense_15/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>

###5.2.2 Parameter Initialization¶

####5.2.2.1 Built-in Initialization

In [17]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4, activation=tf.nn.relu,
        kernel_initializer=tf.random_normal_initializer(mean=0, stddev=0.01),
        bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(1)])

net(X)
net.weights[0], net.weights[1]

(<tf.Variable 'dense_19/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[ 0.00296463,  0.00149156, -0.00837223, -0.0171094 ],
        [-0.01348323,  0.01675946, -0.01125695, -0.01099824],
        [ 0.00835362,  0.0020776 ,  0.00647007,  0.00105665],
        [ 0.01050581, -0.00645115, -0.00026975, -0.0032413 ]],
       dtype=float32)>,
 <tf.Variable 'dense_19/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

In [18]:
# we can also initialize all the parameters to a given constant value (say, 1).

net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4, activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.Constant(1),
        bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(1),
])

net(X)
net.weights[0], net.weights[1]

(<tf.Variable 'dense_21/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=float32)>,
 <tf.Variable 'dense_21/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

In [19]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4,
        activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.GlorotUniform()),
    tf.keras.layers.Dense(
        1, kernel_initializer=tf.keras.initializers.Constant(42)),
])

net(X)
print(net.layers[1].weights[0])
print(net.layers[2].weights[0])

<tf.Variable 'dense_23/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[-0.6986328 ,  0.24206501,  0.63731354, -0.3419237 ],
       [ 0.16383749, -0.52008694,  0.32349032,  0.34888035],
       [ 0.8466193 , -0.08187187, -0.45205438,  0.7080644 ],
       [ 0.7400482 , -0.11892492,  0.7197996 , -0.6592711 ]],
      dtype=float32)>
<tf.Variable 'dense_24/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[42.],
       [42.],
       [42.],
       [42.]], dtype=float32)>


####5.2.2.2 Custom Initialization

In [20]:
class MyInit(tf.keras.initializers.Initializer):
    def __call__(self, shape, dtype=None):
        data=tf.random.uniform(shape, -10, 10, dtype=dtype)
        factor=(tf.abs(data) >= 5)
        factor=tf.cast(factor, tf.float32)
        return data * factor

net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4,
        activation=tf.nn.relu,
        kernel_initializer=MyInit()),
    tf.keras.layers.Dense(1),
])

net(X)
print(net.layers[1].weights[0])

<tf.Variable 'dense_25/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[-6.5818453, -0.       ,  0.       ,  0.       ],
       [-0.       , -0.       , -9.129087 , -0.       ],
       [ 0.       , -0.       ,  7.555254 , -8.114645 ],
       [-0.       ,  7.8406334, -0.       , -0.       ]], dtype=float32)>


In [21]:
net.layers[1].weights[0][:].assign(net.layers[1].weights[0] + 1)
net.layers[1].weights[0][0, 0].assign(42)
net.layers[1].weights[0]

<tf.Variable 'dense_25/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[42.      ,  1.      ,  1.      ,  1.      ],
       [ 1.      ,  1.      , -8.129087,  1.      ],
       [ 1.      ,  1.      ,  8.555254, -7.114645],
       [ 1.      ,  8.840633,  1.      ,  1.      ]], dtype=float32)>

###5.2.3 Tied Parameters

In [22]:
# tf.keras behaves a bit differently. It removes the duplicate layer
# automatically
shared = tf.keras.layers.Dense(4, activation=tf.nn.relu)
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    shared,
    shared,
    tf.keras.layers.Dense(1),
])

net(X)
# Check whether the parameters are different
print(len(net.layers) == 3)

True


##5.3 Deferred Initialization

###5.3.1 Instantiating a Network

In [23]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])

In [24]:
[net.layers[i].get_weights() for i in range(len(net.layers))]

[[], []]

In [25]:
X = tf.random.uniform((2, 20))
net(X)
[w.shape for w in net.get_weights()]

[(20, 256), (256,), (256, 10), (10,)]

##5.4 Custom Layers

###5.4.1 Layers without Parameters

In [26]:
class CenteredLayer(tf.keras.Model):
    def __init__(self):
        super().__init__()

    def call(self, inputs):
        return inputs - tf.reduce_mean(inputs)

In [27]:
layer = CenteredLayer()
layer(tf.constant([1, 2, 3, 4, 5]))

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([-2, -1,  0,  1,  2], dtype=int32)>

In [28]:
net = tf.keras.Sequential([tf.keras.layers.Dense(128), CenteredLayer()])

In [29]:
Y = net(tf.random.uniform((4, 8)))
tf.reduce_mean(Y)

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

###5.4.2 Layers with Parameters

In [30]:
class MyDense(tf.keras.Model):
    def __init__(self, units):
        super().__init__()
        self.units = units

    def build(self, X_shape):
        self.weight = self.add_weight(name='weight',
            shape=[X_shape[-1], self.units],
            initializer=tf.random_normal_initializer())
        self.bias = self.add_weight(
            name='bias', shape=[self.units],
            initializer=tf.zeros_initializer())

    def call(self, X):
        linear = tf.matmul(X, self.weight) + self.bias
        return tf.nn.relu(linear)

In [31]:
dense = MyDense(3)
dense(tf.random.uniform((2, 5)))
dense.get_weights()

[array([[-0.0440588 , -0.01674656, -0.05260754],
        [ 0.03013676, -0.05645331, -0.04577634],
        [ 0.00852349, -0.02807342,  0.12054452],
        [ 0.00520554,  0.00691251, -0.0195082 ],
        [-0.04627716,  0.04016576, -0.04176384]], dtype=float32),
 array([0., 0., 0.], dtype=float32)]

In [32]:
dense(tf.random.uniform((2, 5)))

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.        , 0.        , 0.02161668],
       [0.        , 0.        , 0.        ]], dtype=float32)>

In [33]:
net = tf.keras.models.Sequential([MyDense(8), MyDense(1)])
net(tf.random.uniform((2, 64)))

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.05640336],
       [0.03541207]], dtype=float32)>

##5.5 File I/O

###5.5.1 Loading and Saving Tensors

In [34]:
import numpy as np

x = tf.range(4)
np.save('x-file.npy', x)

In [35]:
x2 = np.load('x-file.npy', allow_pickle=True)
x2

array([0, 1, 2, 3], dtype=int32)

In [36]:
y = tf.zeros(4)
np.save('xy-files.npy', [x, y])
x2, y2 = np.load('xy-files.npy', allow_pickle=True)
(x2, y2)

(array([0., 1., 2., 3.]), array([0., 0., 0., 0.]))

In [37]:
mydict = {'x': x, 'y': y}
np.save('mydict.npy', mydict)
mydict2 = np.load('mydict.npy', allow_pickle=True)
mydict2

array({'x': <tf.Tensor: shape=(4,), dtype=int32, numpy=array([0, 1, 2, 3], dtype=int32)>, 'y': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>},
      dtype=object)

###5.5.2 Loading and Saving Model Parameters

In [38]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.hidden = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(units=10)

    def call(self, inputs):
        x = self.flatten(inputs)
        x = self.hidden(x)
        return self.out(x)

net = MLP()
X = tf.random.uniform((2, 20))
Y = net(X)

In [39]:
net.save_weights('mlp.params')

In [40]:
clone = MLP()
clone.load_weights('mlp.params')

In [41]:
Y_clone = clone(X)
Y_clone == Y

<tf.Tensor: shape=(2, 10), dtype=bool, numpy=
array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]])>

##5.6 GPUs

In [42]:
!nvidia-smi

Fri Apr 29 13:33:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W /  70W |    432MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

###5.6.1 Computing Devices

In [43]:
tf.device('/CPU:0'), tf.device('/GPU:0'), tf.device('/GPU:1')

(<tensorflow.python.eager.context._EagerDeviceContext at 0x7f07443f4230>,
 <tensorflow.python.eager.context._EagerDeviceContext at 0x7f0744449a50>)

In [44]:
len(tf.config.experimental.list_physical_devices('GPU'))

1

In [46]:
def try_gpu(i=0):  #@save
    """Return gpu(i) if exists, otherwise return cpu()."""
    if len(tf.config.experimental.list_physical_devices('GPU')) >= i + 1:
        return tf.device(f'/GPU:{i}')
    return tf.device('/CPU:0')

def try_all_gpus():  #@save
    """Return all available GPUs, or [cpu(),] if no GPU exists."""
    num_gpus = len(tf.config.experimental.list_physical_devices('GPU'))
    devices = [tf.device(f'/GPU:{i}') for i in range(num_gpus)]
    return devices if devices else [tf.device('/CPU:0')]

try_gpu(), try_gpu(10), try_all_gpus()

(<tensorflow.python.eager.context._EagerDeviceContext at 0x7f0744498910>,
 [<tensorflow.python.eager.context._EagerDeviceContext at 0x7f0744498dc0>])

###5.6.2 Tensors and GPUs

In [47]:
x = tf.constant([1, 2, 3])
x.device

'/job:localhost/replica:0/task:0/device:GPU:0'

####5.6.2.1 Storage on the GPU

In [48]:
with try_gpu():
    X = tf.ones((2, 3))
X

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 1., 1.],
       [1., 1., 1.]], dtype=float32)>

In [49]:
with try_gpu(1):
    Y = tf.random.uniform((2, 3))
Y

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.00960255, 0.3304013 , 0.713405  ],
       [0.08699536, 0.35020626, 0.37553167]], dtype=float32)>

####5.6.2.2 Copying

In [50]:
with try_gpu(1):
    Z = X
print(X)
print(Z)

tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1.]], shape=(2, 3), dtype=float32)


In [51]:
Y + Z

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1.0096025, 1.3304013, 1.713405 ],
       [1.0869954, 1.3502063, 1.3755317]], dtype=float32)>

In [52]:
with try_gpu(1):
    Z2 = Z
Z2 is Z

True

###5.6.3 Neural Networks and GPUs

In [53]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    net = tf.keras.models.Sequential([
        tf.keras.layers.Dense(1)])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [54]:
net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.0971088],
       [2.0971088]], dtype=float32)>

In [55]:
net.layers[0].weights[0].device, net.layers[0].weights[1].device

('/job:localhost/replica:0/task:0/device:GPU:0',
 '/job:localhost/replica:0/task:0/device:GPU:0')